In [3]:
from dysh.spectra.spectrum import Spectrum
import numpy as np
from scipy.stats import norm
import astropy.units as u
import matplotlib.pyplot as plt
from dysh.fits.sdfitsload import SDFITSLoad
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.spectra.scan import PSScan, GBTPSScan
import pandas
from copy import deepcopy
pandas.set_option('display.max_rows', 100)
from astropy.modeling.fitting import LevMarLSQFitter,LinearLSQFitter
from specutils import Spectrum1D, SpectrumList,SpectralRegion
from astropy.nddata import StdDevUncertainty
from specutils.fitting import fit_continuum
import matplotlib.pyplot as plt
from dysh.spectra import dcmeantsys,region_to_axis_indices
from dysh.plot.specplot import SpectrumPlot
from astropy.io import fits
from astropy.table import Table
import pandas as pd
#%matplotlib notebook

# 1. Create a gaussian spectrum with noise, compute the baseline, remove it, then add the baseline back in

### First, create the spectrum

In [ ]:
npts = 1000
x = np.arange(npts)
restfreq = 110.20137*u.GHz
cdelt1=10*u.kHz
xfreq = restfreq+(x-npts//2)*cdelt1
y = 3.14+ np.random.rand(npts)+1000*norm(loc=npts/2,scale=50).pdf(x)
header={
'CDELT1': cdelt1.to('Hz').value, 
'CRVAL1': xfreq[0].to('Hz').value,
'CUNIT1': 'Hz',
'CTYPE1': 'FREQ', 'CRPIX1': 1, 'RESTFRQ': restfreq.value, 
'CTYPE2': 'RA--SIN', 'CRVAL2': 221.34, 'CRPIX2': 1,
'CTYPE3': 'DEC--SIN', 'CRVAL3': -12.13, 'CRPIX3': 1,
'CUNIT2': 'deg', 'CUNIT3':'deg',
'NAXIS1': len(x), 'NAXIS2':1, 'NAXIS3':1,
'OBJECT':"NGC 3421"
}

s = Spectrum(flux=y*u.K,spectral_axis=xfreq, rest_value = restfreq, meta=header,
             velocity_convention='radio',radial_velocity=25*u.Unit('km/s'))
s.velocity

In [ ]:
s.write('/tmp/spec.fits',overwrite=True)
#hdu=fits.open('/tmp/spec.fits')
from astropy.nddata import CCDData
s.write.list_formats()
from specutils.io.registers import identify_spectrum_format
identify_spectrum_format('/tmp/spec.fits')

In [ ]:
s.uncertainty=StdDevUncertainty(np.random.rand(npts))

In [ ]:
s.write('/tmp/spec.txt',format='ipac',overwrite=True)

### Plot it with SpectrumPlot

In [ ]:
p = SpectrumPlot(s)
p.plot(grid=True)

In [ ]:
if False:  # testing how to indicate baseline exclude region with a line under the x axis
    p.plot(color='green',grid=True,xaxis_unit="channel",clip_on=False)
    #p.axis.set_clip_on(False)
    xr = p.axis.get_xlim()
    yr = p.axis.get_xlim()
    transx = p.axis.get_xaxis_transform()
    transy = p.axis.get_yaxis_transform()
    p.axis.plot(xr,[-0.02,-0.02], color="orange", transform=transx, clip_on=False,lw=1.2)
    #p.axis.axhline(y=2,color='orange')

### Note plot attributes are sticky, e.g. the grid stays on until you turn it off
Is this a good feature?  Would be useful for a standalone plot that stays up between plot updates.

In [ ]:
p.plot(color='gray',xaxis_unit="km/s")

In [ ]:
p.plot(xaxis_unit="mm",yaxis_unit="mK")
# Haha it should say wavelength not frequency

### Reset arguments to defaults

In [ ]:
p.reset()
p.plot()

### You can also plot directly from the Spectrum

In [ ]:
s.plot(color='green',grid=True,xaxis_unit="channel")

----------------------

## Baseline subtraction


### Use channels for the exclude region. Compute the baseline, exclude channels 300 to 700, don't remove it.
The red line shows the computed baseline
Note that since exclude_action is not set, the exclude region attribute on the spectrum itself will not be set.
Attempting to show baseline exlcusion range with orange line.  Seems shifted -- is it radial_velocity?

In [ ]:
s._plotter=None
s._undo_baseline()
s.baseline(1,exclude=[300,700],remove=False)#,exclude_action='replace')
s.plot()
print(s.exclude_regions)
idx = region_to_axis_indices(s.exclude_regions[0],s)
print(idx)
s.bshow()
s.plotter.axis.plot(s.spectral_axis,s._baseline_model(s.spectral_axis),color='r')
transx = p.axis.get_xaxis_transform()
xr1 =(s.spectral_axis[0].value,s.spectral_axis[idx[0]].value)
print("XLIM ",s.plotter.axis.get_xlim())
print(xr1)
xr2 =(s.spectral_axis[idx[1]].value,s.spectral_axis[-1].value)
print(xr2)
s.plotter.axis.plot(xr1,[-0.02,-0.02], color="orange", transform=transx, clip_on=False,lw=1.2)
s.plotter.axis.plot(xr2,[-0.02,-0.02], color="orange", transform=transx, clip_on=False,lw=1.2)
#ax.plot(s.spectral_axis,s._baseline_model(s.exclude_regions),color='g')
#ax.axvline(x=110.201,ymin=0,ymax=12,color='k',lw=4)

## test using exclude outside spectral axis
first should warn, second should except


In [ ]:
s.baseline(1,exclude=[300,6700],remove=False,fix_exclude=True)
s.baseline(1,exclude=[300,6700],remove=False)

## Use a frequency Quantity for the exclude region
Try presetting a spectral exclude region and replace it or not.

In [ ]:
s._exclude_regions = [SpectralRegion(110.199*u.GHz,110.2037*u.GHz)]
s.baseline(degree=0,remove=False)
s.bshow()
s.baseline(degree=1,remove=True,exclude=[110.198*u.GHz,110.204*u.GHz],exclude_action='replace')
s.plot(color='black',xaxis_unit="km/s")
s.bshow()
# haha should say "Velocity"

In [ ]:
# this should except
s.baseline(1,exclude=[110.199*u.GHz,210.2037*u.GHz],remove=False)

In [ ]:
s._undo_baseline()
kms = u.Unit("km/s")
s.baseline(2,exclude=[19.559188*kms,30.4408*kms],remove=False)
s.plot()
s.plotter.axis.plot(s.spectral_axis,s._baseline_model(s.spectral_axis),color='r')

### print some statistics [mean, rms, min, max], show the baseline model function

In [ ]:
print(s.stats())
s.bshow()

### now do again it and remove the baseline

In [ ]:
### undo the baseline removal.  Stats shows it has been added back in and the spectrum is restored

In [ ]:
s._undo_baseline()
print(s.stats())
s.bshow()


In [2]:
from dysh.fits.gbtfitsload import GBTFITSLoad
# Assuming you are in the dysh root dir.
sdf_file = "/home/mpound/src/dysh/src/dysh/fits/tests/data/TGBT21A_501_11_ifnum_0_int_0-2.fits"
sdf = GBTFITSLoad(sdf_file)
psscan = sdf.getps(152, plnum=0)
psscan.calibrate()
psscan.tsys
#>>> array([17.17140405, 17.3474724 , 17.0515161 ])
# The above should be:
#>>> array([17.24000345, 17.17140405, 17.15663698])

FileNotFoundError: [Errno 2] No such file or directory: '/home/mpound/src/dysh/src/dysh/fits/tests/data/TGBT21A_501_11_ifnum_0_int_0-2.fits'

In [4]:
datadir = '/data/gbt'
#f = datadir + '/examples/misc/ngc5291.fits'
f = datadir + '/examples/onoff-L/data/TGBT21A_501_11.raw.vegas.fits'
#f = datadir + 'examples/nod-KFPA/data/TGBT22A_503_02.raw.vegas/TGBT22A_503_02.raw.vegas.A.fits'
#f = datadir + '/examples/mixed-fs-ps/data/AGBT16B_225_05/AGBT16B_225_05.raw.vegas/AGBT16B_225_05.raw.vegas.A.fits'
p = GBTFITSLoad(f)

In [6]:
psscan = p.getps(152, plnum=0)
psscan.calibrate()
psscan.tsys

PSSCAN nrows = 302


array([17.24000345, 17.17140405, 17.15663698, 16.85934981, 17.35722692,
       17.13173653, 17.21543606, 17.30092163, 17.00566475, 16.96166605,
       16.90800304, 17.25126367, 16.95205031, 17.07164545, 17.31209165,
       17.24198654, 17.23042795, 17.23062501, 17.22081776, 17.1646568 ,
       17.10683608, 17.06225591, 17.00860262, 16.95643422, 16.86738768,
       17.24130516, 17.23751937, 17.08227286, 17.39648505, 17.25251959,
       17.07859948, 17.07804021, 17.17861759, 17.24824253, 17.24999249,
       16.90474598, 16.96176874, 17.00971976, 17.08731873, 17.0157204 ,
       17.18919643, 17.00773527, 17.22850453, 17.12418577, 17.33776924,
       16.96650514, 17.35763214, 17.11744129, 17.39144335, 17.39013886,
       17.05785951, 17.19779633, 17.23384321, 17.30106595, 17.30310179,
       17.24598466, 17.0762514 , 16.90833193, 17.10749943, 17.27291816,
       17.28188999, 16.98815547, 17.00188728, 17.13296885, 17.07062684,
       17.27227147, 17.30230105, 17.07988731, 16.8963695 , 17.10